## The  Problem
Use the SNP-data below to show equivalent results of G-BLUP (Van Radens method 1) with SNP-BLUP. Model only the random part of the MME. Assume phenotypes for 5 animals being 1, 2, 3, 4, 5 and a heritability of 0.5. Consider the following genotypes coded (0,1,2) for the 5 animals (5 rows) and 10 SNPs (10 columns): 

```
2 0 2 1 1 0 2 0 0 1
0 0 0 2 2 1 1 0 1 1
2 2 0 1 0 0 1 2 2 2
0 0 0 0 1 2 1 2 1 0
0 2 1 2 1 2 0 1 1 0
```
<!-- pandoc -o mk-blup.pdf --pdf-engine=xelatex -V monofont='DejaVu Sans Mono' mk-blup.ipynb -->

In [ ]:
# Packages and data initialization
using LinearAlgebra, Statistics
M = [
     2 0 2 1 1 0 2 0 0 1
     0 0 0 2 2 1 1 0 1 1
     2 2 0 1 0 0 1 2 2 2
     0 0 0 0 1 2 1 2 1 0
     0 2 1 2 1 2 0 1 1 0
    ]
h² = 0.5
y = [1, 2, 3, 4, 5]
nid, nlc = 5, 10
X = ones(nid);

In [ ]:
# Genotype manipulations
p = mean(M, dims = 1) # as the dimensions are `nID by nLoci`, the first dim is nID, average on them
tpq = 2p' * (1 .- p); tpq = tpq[1]
W = M .- 2p
λ = (1 - h²) / h² * nlc  # also suggested tpq for nlc

## SNP-BLUP
### The model
$$\mathbf{y} = \mu + \mathbf{{\color{cyan}Wb} + e}$$

where,

- $\mathbf{y}$: phenotypes, here 1:5
- $\mathbf{W}$: genotypes that link SNP effects to phenotypes, here is the corrected ones
- $\mathbf{b}$: SNP effects to be estimated
- $\mathbf{e}$: residual effects

### The MME
$$\begin{bmatrix}\mathbf{X'X} & \mathbf{X'W}\\\mathbf{W'X} & \mathbf{W'W} + \lambda\mathbf{I}\end{bmatrix}\hat{\mathbf{b}} = \begin{bmatrix}\mathbf{W'y}\\\mathbf{W'y}\end{bmatrix}$$

In [ ]:
# SNP-BLUP
lhs = [X'X X'W
       W'X W'W + λ * I]

rhs = [X'y; W'y]
bhat = lhs \ rhs
gebv1 = W * bhat[2:end]

In [ ]:
# how good is this
cor(gebv1, y)

## GBLUP
### The model
$$\mathbf{y} = \mu + \mathbf{Za + e}$$

where:
- $\mathbf{Z}$: the incidence matrix for individuals with phentoypes, $\mathbf{I}$ here.

The individuals are related with GRM $\mathbf{G}$.

### The MME
$$\begin{bmatrix}\mathbf{X'X} & \mathbf{X'Z}\\\mathbf{Z'X} & \mathbf{Z'Z} + \lambda \mathbf{G}^{-1}\end{bmatrix}\hat{\mathbf{a}}=\begin{bmatrix}\mathbf{X'y}\\\mathbf{y}\end{bmatrix}$$

In [ ]:
Z = I(nid)
grm = W * W' / tpq + 0.01I
giv = inv(grm)
λ2 = (1 - h²) / h²
lhs = [X'X X'Z
       Z'X Z'Z + λ*giv]
rhs = [X'y; y]
ahat = lhs \ rhs
gebv2 = ahat[2:end]

In [ ]:
# how good it is
cor(y, gebv2)

In [ ]:
# apprx. equivalence
cor(gebv1, gebv2)